In [ ]:
#First Step
pip install jupyter-dash pyngrok -q
pip install dash
pip install dash-bootstrap-components

#If it appears an error when trying to install all at the same time, try to install each line separately

In [ ]:
import sys
print(sys.version)
#Just to check python version

In [ ]:
from jupyter_dash import JupyterDash
import dash
import pandas as pd
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
from dash_bootstrap_templates import load_figure_template

#If something gives error, try to install the line that gives error in another cell individually

In [ ]:
import os
os.chdir(r"\\Mac\Home\Documents\Datathon")

#importando base de datos 
products = pd.read_csv("products.csv", sep=",", encoding="latin-1")
categories = pd.read_csv("products_categories.csv", sep=",",encoding="latin-1", on_bad_lines="skip")
items = pd.read_csv("items_ordered_2years.txt", sep="|", on_bad_lines="skip")

LIMPIEZA DE DATOS

In [ ]:
len(products.columns)
len(categories.columns)
len(items.columns)

#verificando que el tipo de variables estén correctamente importadas 
products.dtypes
categories.dtypes
items.dtypes

#analizamos df categorías
categories.describe()
print(categories['cat1'].drop_duplicates())
print(categories['cat2'].drop_duplicates())
print(categories['cat3'].drop_duplicates())

#no se han hallado errores en los datos del df de categorías

#analizamos df productos 
products.describe()
unique_names = pd.DataFrame(products['name'].drop_duplicates())
unique_brands = pd.DataFrame(products['marca_value'].drop_duplicates()) 

#analizamos df items
items.describe()
unique_cities = pd.DataFrame(items['city'].drop_duplicates())
#ponemos en minúsculas los nombres de las ciudades
items['city']=items['city'].str.lower() 

#vemos si hay errores en columnas "price" - "qty_ordered" - "discount percent"
items['price'].dtypes
items[items['price'].isnull()]
items[items['qty_ordered'].isnull()]
items[items['discount_percent'].isnull()]
#no hay ningún error 

#creando nueva df para desarrollo de HeatMap
dfheat = pd.DataFrame(items)
dfheat['Total_sales'] = (dfheat['price']*dfheat['qty_ordered'])*(1-dfheat['discount_percent']/100)
dfheat = dfheat.drop(['qty_ordered','base_cost','price','discount_percent','num_order','item_id','created_at','customer_id'], axis=1)
#eliminando todas las ventas de valor 0 
dfheat = dfheat[dfheat['Total_sales']!=0]



In [ ]:
print(type(dfheat.iloc[81369,2]))
#reseteando indexación de filas 
dfheat.reset_index()

#determinando CPs donde hay nombres de ciudad
s3 = pd.Series(dfheat['zipcode'])
s3 = s3.str.replace("-","")
s3 = s3.str.replace(" ","")
dfheat['NumOrNot'] = s3.str.isnumeric()
#df de zipcodes con nombres de ciudad 
error_ciudades = pd.DataFrame(dfheat[dfheat['NumOrNot']==False])
#hacemos que dfheat solo tenga valores correctos  dado que luego se les añadirán corregidos 
dfheat = dfheat[dfheat['NumOrNot']==True]

Corrección de códigos postales con ciudades

In [ ]:
#exportamos tabla con errores para corregir manualmente por medio de Excel
#importamos tabla ya corregida 
correct_error = pd.read_csv('tabla_correcciones.csv', sep=',', encoding='utf-8')
correct_error.head(3)

#eliminamos columnas no necesarias 
dfheat = dfheat.drop(['product_id','city','NumOrNot'], axis=1)
correct_error = correct_error.drop(['city'],axis=1)


In [ ]:
#concatenamos datos con tabla dfheat 
dfheat_Final = pd.concat([dfheat,correct_error]).reset_index()


In [ ]:
#agrupamos total de ventas por codigo postal
dfheat_Final['zipcode']=dfheat_Final['zipcode'].astype(str)
print(dfheat_Final.dtypes)
df_aggregate = dfheat_Final.groupby(['zipcode'],as_index=False)['Total_sales'].sum()


In [ ]:
#unimos tabla de ventas totales con tabla de códigos postales externa 
zipcodes = pd.read_csv('zipcodes_SP_PR_UK.csv', sep=',', encoding='utf-8')
mergedHeat = pd.merge(df_aggregate,zipcodes, on='zipcode',how='inner')


ANALISIS EXPLORATORIO DE LOS DATOS

In [ ]:
items["venta"] = items["price"]*items["qty_ordered"]*(1-(items["discount_percent"]/100))
items['fecha'] = pd.to_datetime(items['created_at']).dt.date
type(items['fecha'])
items['fecha'] = pd.to_datetime(items['fecha'], format="%Y/%m/%d")
df = items.groupby(items['fecha']).sum()
df = df.reset_index()

def drawFigure1():
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                dcc.Graph(
                    figure = go.Figure(go.Scatter(x=list(df.fecha), y=list(df.venta)), layout = go.Layout(
                        template='plotly_dark',
                        plot_bgcolor= 'rgba(0, 0, 0, 0)',
                        paper_bgcolor= 'rgba(0, 0, 0, 0)',
                        title_font_color="#D5D5E2",
                        xaxis=dict(
                            rangeselector=dict(
                                buttons=list([
                                    dict(count=1,
                                        label="1m",
                                        step="month",
                                        stepmode="backward"),
                                    dict(count=6,
                                        label="6m",
                                        step="month",
                                        stepmode="backward"),
                                    dict(count=1,
                                        label="YTD",
                                        step="year",
                                        stepmode="todate"),
                                    dict(count=1,
                                        label="1y",
                                        step="year",
                                        stepmode="backward"),
                                    dict(step="all")
                                ])
                            ),
                            rangeslider=dict(
                                visible=True
                            ),
                            type="date"
                        ))),
                    config={
                        'displayModeBar': False}
                ), 
            ])
        )
    ])

In [ ]:
def drawText1():
    return html.Div([
        dbc.Card(
                dbc.CardBody(
                    [ html.Div([ # second column on second row
                    html.H4('Ventas a traves del tiempo', className='card-title'),
                    html.P("El presente grafico muestra como varian las ventas de Atmira desde el 01/01/2017 "
                                     "hasta el 31/12/2018. Con este grafico, se observa que en los meses "
                                     "de mayo y noviembre es donde se registra una mayor cantidad de ventas. Esto puede ocurrir debido al cambio de temporada, "
                                     "que puede causar resfriados en las personas y por ende incrementar el consumo de medicamentos. Otra posibilidad seria que durantes "
                                     "estos tres picos, se haya implementado una campaña de marketing lo cual cause un incremento muy alto durante lapsos pequeños. "
                                     " De igual manera, es evidente que a medida que pasa el tiempo, las ventas incrementan, lo cual nos indica "
                                     "una mejor aceptación por parte del mercado a medida que la farmacia adquiere popularidad.",className="card-text",
                                     ),
                    ], style = {'textAlign': 'left'})
                    ]
            ), 
            )
    ])

In [ ]:
#Encontrar el total de ventas
df_2 = items.groupby(items['product_id']).sum()
print(df_2.head(10))
v1 = print(df_2["venta"].sum())

import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
rev = locale.currency(11583565.12561113, grouping = True)
print(rev)

In [ ]:
#Encontrar la media de ventas por mes
items["created_at"] = pd.to_datetime(items["created_at"], format='%Y/%m/%d %H:%M:%S')
type(items["created_at"])
items["created_at"]

items["mes"] = pd.to_datetime(items["created_at"]).dt.month
print(items["mes"])

pm = (items.iloc[:,[-1,-3]])
print(pm.head())
pm_1 = pm["venta"].groupby(pm['mes']).sum()
print(pm_1)

m2 = print(pm_1.mean())

import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
month = locale.currency(965297.0938009274, grouping = True)
print(month)

In [ ]:
# Grafico 2: Promedio de ventas por hora del día
# Codigo con calculos
items['Hora'] = "x"

items["created_at"] = pd.to_datetime(items["created_at"], format='%Y/%m/%d %H:%M:%S')
type(items["created_at"])
items["created_at"]

for i in range(0,len(items)):
    items['Hora'][i]=items['created_at'][i].hour

H = items["venta"].groupby(items['Hora']).mean()
H = H.reset_index()


In [ ]:
#Para evitar perdida de tiempo importamos el dataframe ya hecho
H = pd.read_csv("porhora.csv", sep=",", encoding="latin-1")
fi = px.bar(H, y="venta", x="Hora", template="plotly")

# Set layout
fi.update_layout(plot_bgcolor='rgba(30, 41, 50, 0.5)',paper_bgcolor='rgba(0, 0, 0, 0.3)')

# Plot size
fi.update_layout(
    width=800,
    height=600,
    autosize=True,
    margin=dict(t=10, b=5, l=5, r=0),
    template = "plotly_dark")

In [ ]:
#GRAFICO 2
#Promedio de ventas por hora
H = pd.read_csv("porhora.csv", sep=",", encoding="latin-1")

def drawFigure2():
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                dcc.Graph(
                    figure = px.bar(H, y="venta", x="Hora", template="plotly", 
                    title = "Promedio de ventas por hora").update_layout(
                        template='plotly_dark',
                        plot_bgcolor= 'rgba(0, 0, 0, 0)',
                        paper_bgcolor= 'rgba(0, 0, 0, 0)',
                        title_font_color="#D5D5E2",
                    ),
                    config={
                        'displayModeBar': False }
                )
            ])
        )
    ])


In [ ]:
def drawText2():
    return html.Div([
        dbc.Card(
                dbc.CardBody(
                    [ html.Div([ # second column on second row
                html.H4('Promedio de ventas por hora', className='card-title'),
                html.P("En cuanto al gráfico del promedio de ventas por hora del día podemos evidenciar que  en las primeras horas del dia "
                        "vemos un crecimiento de las ventas, el cual llega a un máximo a las 6:00 y después baja un poco y se mantiene relativamente constante hasta 20:00. "
                        "Por otro lado, se destaca tambien un continuo decremento de ventas "
                        "a partir de las 20:00 lo cual indicaria que a partir de esa hora es poca la afluencia de clientes "
                        "que realizan ordenes. Finalmente, este gráfico de barras tambien nos muestra que las horas pico "
                        "son tanto en la mañana como en la tarde. Esto puede darnos una idea de que la gente compra mas cuando "
                        "va al trabajo por la mañana o regresa a casa por la tarde, aunque habria que analizar la correlación "
                        "entre estas variables.",className="card-text",
                                     ),
                    ], style = {'textAlign': 'left'})
                    ]
            ), 
            )
    ])

In [ ]:
#GRAFICO 3
# Ventas por categoria
it = pd.merge(items, products, on = "product_id")
itgr = it["venta"].groupby(it['analytic_category']).sum()
itgr = itgr.reset_index()

def drawFigure3():
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                dcc.Graph(
                    figure = px.bar(itgr, y='venta', x='analytic_category', labels = {'analytic_category': "Categoría", "venta":"Ventas (miles)"}, 
                    title = "Ventas por categoría").update_layout(
                        template='plotly_dark',
                        plot_bgcolor= 'rgba(0, 0, 0, 0)',
                        paper_bgcolor= 'rgba(0, 0, 0, 0)',
                        title_font_color="#D5D5E2",
                    ),
                    config={
                        'displayModeBar': False }
                )
            ])
        )
    ])

In [ ]:
def drawText3():
    return html.Div([
        dbc.Card(
                dbc.CardBody(
                    [ html.Div([ # second column on second row
                html.H4('Ventas por Categoría 1', className='card-title'),
                html.P("El grafico de ventas por categoría 1 nos muestra las subcategorías de categoría 1 con "
                "sus respectivas ventas. A partir del grafico llegamos a determinar que la subcategoría con más ventas es " 
                "cosmética y belleza, con 7.85 millones de euros aproximadamente, mientras que las categorías que generan menos ingresos son la de veterinaria y perfumería. "
                "Con estos datos, podriamos realizar una recomendación "
                "acerca de que la categoría a la cual la farmacia debería darle más importancia por su volumen de ventas.",className="card-text",
                                     ),
                    ], style = {'textAlign': 'left'})
                    ]
            ), 
            )
    ])

In [ ]:
#Grafico 4
#Heatmap

def drawFigure4():
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                dcc.Graph(
                    figure = px.density_mapbox(mergedHeat, lat='latitude', lon='longitude', z='Total_sales', radius=10,
                        center=dict(lat=35.2842, lon=-3), zoom=3,
                        mapbox_style="stamen-terrain", 
                        title = "Ventas por categoría").update_layout(
                        template='plotly_dark',
                        plot_bgcolor= 'rgba(0, 0, 0, 0)',
                        paper_bgcolor= 'rgba(0, 0, 0, 0)',
                        title_font_color="#D5D5E2",
                    ),
                    config={
                        'displayModeBar': False }
                )
            ])
        )
    ])


In [ ]:
def drawText4():
    return html.Div([
        dbc.Card(
                dbc.CardBody(
                    [ html.Div([ # second column on second row
                html.H4('Heatmap de ventas', className='card-title'),
                html.P("Por medio del mapa de calor, podemos llegar a conclusiones más generales y que nos "
                "ayudan a darnos una idea de la dispersión geográfica de los clientes. Podemos claramente "
                "ver que hay un mayor número de ventas en los alrededores de las ciudades más grandes "
                "en específico en Madrid, Bilbao, Valencia y Oviedo. Debemos de tener en consideración "
                "que también han tenido ventas en otros países como Portugal y el Reino Unido, sin "
                "embargo sus niveles de ventas son muy pequeños en relación al total de ventas dentro del "
                "territorio español. Podemos concluir a plena vista, que la empresa sí tiene un mercado muy "
                "diversificado dentro de España y que esto puede ser visto como una fortaleza dado que su "
                "cartera de clientes es bastante amplia y esto puede ayudar a mitigar efectos negativos de "
                "políticas regionales en España.",className="card-text",
                                     ),
                    ], style = {'textAlign': 'left'})
                    ]
            ), 
            )
    ])

In [ ]:
load_figure_template("cyborg")

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.CYBORG])

# LAYOUT
app.layout = dbc.Container(
    [
        dbc.Row(dbc.Col(html.H2('Atmira Pharma Visualization', className='d-flex justify-content-center', 
                                style = {'max-width': '100%','color':"rgb(82, 97, 211)"}))),  # header row
    
        dbc.Row([  # start of second row
            dbc.Col([  # first column on second row
                dbc.Card(
                [ dbc.Row(
                    [
                        dbc.Col( 
                            dbc.CardImg(
                                src="/assets/revenue.png",
                                top=True,
                                style={"width": "6rem"},  #Altura de la imagen
                                className="img-fluid rounded-start",
                            ),
                            className="col-md-4",
                        ),
                        dbc.Col( 
                           dbc.CardBody(
                               [
                                 html.H5("Venta total", className="card-title", style = {'text-transform': "uppercase"}),
                                 html.H4(rev, className = "card-title", style = {"color": "rgb(122, 67, 182)"})
                               ]
                           ),
                           className="col-md-8", 
                        ),
                    ], className="g-0 d-flex align-items-center"
                )
            ], 
            className="mb-3",
            style={
                "maxWidth": "600px",
                'backgroundColor': 'rgb(44, 48, 52)'},
            color = "light", outline =True),
            ], 
            width={'size': 4, 'offset': 0, 'order': 1}),  # width second column on first row
            
            dbc.Col([  # Second column on first row
                dbc.Card(
                [ dbc.Row(
                    [
                        dbc.Col(
                            dbc.CardImg(
                                src="/assets/revenue.png",
                                top=True,
                                style={"width": "6rem"},  #Altura de la imagen
                                className="img-fluid rounded-start",
                            ),
                            className="col-md-4",
                        ),
                        dbc.Col(
                           dbc.CardBody(
                               [
                                 html.H5("Venta Media Por Mes", className="text-center", style = {'text-transform': "uppercase"}),
                                 html.H4(month, className = "card-title", style = {"color": "rgb(122, 67, 182)"})
                               ]
                           ),
                           className="col-md-8", 
                        ),
                    ], className="g-0 d-flex align-items-center"
                )
            ], 
                className="mb-3",
                style={
                "maxWidth": "600px",
                'backgroundColor': 'rgb(44, 48, 52)'},
            color = "light", outline =True),
            ], 
            width={'size': 4, 'offset': 0, 'order': 1}),  # width second column on first row
            
            dbc.Col([  # third column on first row
                dbc.Card(
                [ dbc.Row(
                    [
                        dbc.Col(
                            dbc.CardImg(
                                src="/assets/revenue.png",
                                top=True,
                                style={"width": "6rem"},  #Altura de la imagen
                                className="img-fluid rounded-start",
                            ),
                            className="col-md-4",
                        ),
                        dbc.Col(
                           dbc.CardBody(
                               [
                                 html.H5("Top 3 categorías más vendidas", className="card-title", style = {'text-transform': "uppercase"}),
                                 dbc.ListGroup([
                                    dbc.ListGroupItem("Top 1: Cosmetica y belleza"),
                                    dbc.ListGroupItem("Top 2: Higiene"),
                                    dbc.ListGroupItem("Top 3: Infantil"),
                                 ],
                                 flush = True, style = {"color": "rgb(122, 67, 182)"},
                                ),   
                               ]
                           ),
                           className="col-md-8", 
                        ),
                    ], className="g-0 d-flex align-items-center"
                )
            ], 
            className="mb-3",
            style={
                "maxWidth": "600px",
                'backgroundColor': 'rgb(44, 48, 52)'},
            color = "light", outline =True),
            ], 
            width={'size': 4, 'offset': 0, 'order': 1})  # width third column on first row
        ]),  # end of first row
        
        dbc.Row([  # start of second row
            dbc.Col([  # first column on second row
                drawFigure1()
            ], width=8) ,  # width first column on second row
            dbc.Col([
                drawText1()
            ], width= 4),  # width second column on second row
        ], align = "center") , html.Br(), # end of second row
        
        dbc.Row([ # start of third row
            dbc.Col([
               drawFigure2()
            ], width= 8), 

            dbc.Col([
                drawText2()
            ], width= 4),  
        ], align = "center"), html.Br(),

        dbc.Row([ # start of fourth row
            dbc.Col([
               drawFigure3()
            ], width=8),

            dbc.Col([
                drawText3()
            ], width=4),  
            
        ], align = "center"), html.Br(), 

        dbc.Row([ # start of fourth row
            dbc.Col([
               drawFigure4()
            ], width=8),
             
            dbc.Col([
                drawText4()
            ], width=4), 

        ], align = "center"),

    ], fluid=True, className = "dbc")



In [ ]:
if __name__ == "__main__":
    app.run_server(debug=True, port=8051)